In [11]:
# Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pybaseball import schedule_and_record
from pybaseball import cache
from pybaseball import batting_stats_range
from pybaseball import statcast
import tqdm

cache.enable()

In [15]:
# initialize an empty dataframe
df = pd.DataFrame()

# Loop through each year 2015-2022 as a string and for each year March 15 through November 15; use tqdm to monitor progress
for year in range(2015, 2022):
    for date in tqdm.tqdm(pd.date_range(f"{year}-03-15", f"{year}-11-15").strftime("%Y-%m-%d")):
        try:
            # Retrieve the batting stats for that day
            data = batting_stats_range(date, date)
            # Append the data to the dataframe
            df = df.concat(data)
        except IndexError:
            # If there is no data for that day, skip it
            pass

 12%|█▏        | 30/246 [02:58<21:27,  5.96s/it]


KeyboardInterrupt: 